In [28]:
import pdfplumber
from pdfminer.high_level import extract_text

import io
import requests
import re
import pandas as pd
import datetime

In [29]:
def get_text(url):
    rq = requests.get(url)
    left_column_text_all_pages = ""
    right_column_text_all_pages = ""

    with pdfplumber.open(io.BytesIO(rq.content)) as pdf:
        for page in pdf.pages:
            words = page.extract_words()
            threshold_x = 300

            left_column_words = [word for word in words if word['x0'] < threshold_x]
            right_column_words = [word for word in words if word['x0'] >= threshold_x]
            left_column_text = ' '.join([word['text'] for word in left_column_words])
            right_column_text = ' '.join([word['text'] for word in right_column_words])
            left_column_text_all_pages += left_column_text + ' '
            right_column_text_all_pages += right_column_text + ' '

            left_column_clean = re.sub(r'(?<=\w)-(\s)(?=\w)', r'', left_column_text_all_pages) # remove end_line hyphens
            left_column_clean = re.sub(r'DOC 55 \d{4}/\d{3}', '', left_column_clean)
            left_column_clean = re.sub(r'\d{4}/\d{3} DOC 55', '', left_column_clean)
            left_column_clean = re.sub(r'CHAMBRE \d+e SESSION DE LA 55e LÉGISLATURE \d{4}(?: \d+)?', '', left_column_clean)
            right_column_clean = re.sub(r'(?<=\w)-(\s)(?=\w)', r'', right_column_text_all_pages) # remove end_line hyphens
            right_column_clean = re.sub(r'DOC 55 \d{4}/\d{3}', '', right_column_clean)
            right_column_clean = re.sub(r'\d{4}/\d{3} DOC 55', '', right_column_clean)
            right_column_clean = re.sub(r'\d{4} KAMER • \d+e ZITTING VAN DE 55e ZITTINGSPERIODE(?: \d+)?', '', right_column_clean)

    return left_column_clean, right_column_clean    

In [36]:
df = pd.read_json("./folder_to_ignore/raw_data/New_Documents_Parlementaires_Récents.json").T

In [37]:
df.head()

,page's main url,dossier id,text,date,pdf link,pdf id,type of documents
2657,https://www.lachambre.be/kvvcr/showpage.cfm?se...,2657,Proposition de loi modifiant la loi du 10 nove...,17/08/2023,https://www.lachambre.be/site/wwwcfm/flwb/flwb...,003,AVIS DU CONSEIL D'ETAT
3152,https://www.lachambre.be/kvvcr/showpage.cfm?se...,3152,Projet de loi relatif à l'approche administrat...,27/07/2023,https://www.lachambre.be/site/wwwcfm/flwb/flwb...,012,RAPPORT COMPLEMENTAIRE
3522,https://www.lachambre.be/kvvcr/showpage.cfm?se...,3522,L'évolution de la dette de l'État fédéral - Au...,10/08/2023,https://www.lachambre.be/site/wwwcfm/flwb/flwb...,001,RAPPORT
3514,https://www.lachambre.be/kvvcr/showpage.cfm?se...,3514,Projet de loi droit de la procédure pénale I,10/08/2023,https://www.lachambre.be/site/wwwcfm/flwb/flwb...,001,PROJET DE LOI
3520,https://www.lachambre.be/kvvcr/showpage.cfm?se...,3520,Projet de loi modifiant la loi du 7 mai 1999 p...,10/08/2023,https://www.lachambre.be/site/wwwcfm/flwb/flwb...,001,PROJET DE LOI


In [38]:
df[['fr_text', 'nl_text']] = df['pdf link'].apply(lambda url: pd.Series(get_text(url)))
df.to_csv("processed_doc_recent.csv")


In [42]:
columns = df.columns

In [43]:
columns

Index(['page's main url', 'dossier id', 'text', 'date', 'pdf link', 'pdf id',
       'type of documents', 'fr_text', 'nl_text'],
      dtype='object')

In [45]:
df.T.to_json(r"processed_doc_recent.json")

In [23]:
# def fr_clean(text):
#     clean_text = re.sub(r'(?<=\w)-(\s)(?=\w)', r'', text) # remove end_line hyphens
#     clean_text = re.sub(r'DOC 55 \d{4}/\d{3}', '', clean_text)
#     clean_text = re.sub(r'\d{4}/\d{3} DOC 55', '', clean_text)
#     clean_text = re.sub(r'CHAMBRE \d+e SESSION DE LA 55e LÉGISLATURE \d{4}(?: \d+)?', '', clean_text)
#     return clean_text

# def nl_clean(text):
#     clean_text = re.sub(r'(?<=\w)-(\s)(?=\w)', r'', text) # remove end_line hyphens
#     clean_text = re.sub(r'DOC 55 \d{4}/\d{3}', '', clean_text)
#     clean_text = re.sub(r'\d{4}/\d{3} DOC 55', '', clean_text)
#     clean_text = re.sub(r'\d{4} KAMER • \d+e ZITTING VAN DE 55e ZITTINGSPERIODE(?: \d+)?', '', clean_text)
#     return clean_text

In [25]:
# preprocess_df = pd.read_csv("./processed_doc_recent.csv")
# preprocess_df['fr_text'] = preprocess_df['fr_text'].apply(lambda text: pd.Series(fr_clean(text)))
# preprocess_df['nl_text'] = preprocess_df['nl_text'].apply(lambda text: pd.Series(nl_clean(text)))
# preprocess_df.to_csv("processed_doc_recent.csv")

In [4]:
# df = pd.read_json('apercu_complet.json')
# df_pdf_not_disponible = df[df["DocumentChambre"].str.contains("pasdisponible")]
# df_pdf_disponible = df[~df["DocumentChambre"].str.contains("pasdisponible")]

In [ ]:
# first_300 = df_disponible.iloc[:300]
# first_300[['fr_text', 'nl_text']] = first_300['DocumentChambre'].apply(lambda url: pd.Series(get_text(url)))
# first_300.to_csv("first_300.csv")

# df_pdf_not_disponible["fr_text"] = "PDF not disponible"
# df_pdf_not_disponible["nl_text"] = "PDF not disponible"
# df_pdf_not_disponible.pdf_not_disponible.csv")

In [ ]:
# def new_get_text(df):
#     try:
#         rq = requests.get(df["DocumentChambre"])
#         left_column_text_all_pages = ""
#         right_column_text_all_pages = ""

#         with pdfplumber.open(io.BytesIO(rq.content)) as pdf:
#             for page in pdf.pages:
#                 words = page.extract_words()
#                 threshold_x = 300

#                 left_column_words = [word for word in words if word['x0'] < threshold_x]
#                 right_column_words = [word for word in words if word['x0'] >= threshold_x]

#                 left_column_text = ' '.join([word['text'] for word in left_column_words])
#                 right_column_text = ' '.join([word['text'] for word in right_column_words])

#                 left_column_text_all_pages += left_column_text + ' '
#                 right_column_text_all_pages += right_column_text + ' '

#                 left_column_clean = re.sub(r'(?<=\w)-(\s)(?=\w)', r'', left_column_text_all_pages) # remove end_line hyphens
#                 left_column_clean = re.sub(r'DOC 55 \d{4}/001', '', left_column_clean)
#                 left_column_clean = re.sub(r'\d{4}/001 DOC 55', '', left_column_clean)
#                 left_column_clean = re.sub(r'CHAMBRE \d+e SESSION DE LA 55e LÉGISLATURE \d{4}(?: \d+)?', '', left_column_clean)

#                 right_column_clean = re.sub(r'(?<=\w)-(\s)(?=\w)', r'', right_column_text_all_pages) # remove end_line hyphens
#                 right_column_clean = re.sub(r'DOC 55 \d{4}/001', '', right_column_clean)
#                 right_column_clean = re.sub(r'\d{4}/001 DOC 55', '', right_column_clean)
#                 right_column_clean = re.sub(r'\d{4} KAMER • \d+e ZITTING VAN DE 55e ZITTINGSPERIODE(?: \d+)?', '', right_column_clean)

#         df["fr_text"] = left_column_clean
#         df["nl_text"] = right_column_clean
#         return df

#     except Exception as e:
#         print(type(e))
#         return e

# for i in df_pdf_disponible.loc(): 
#     print(i)
#     # new_get_text(i)

# # search_links = get_search_url_list(end_page=334)

# # urls = list(itertools.chain.from_iterable(thread_map(get_urls_from_search_page, search_links)))

# # with requests.Session() as session:
#     # new_df = df for df in thread_map(partial(new_get_text, session=session), df_pdf_disponible)

# # properties.to_csv("properties.csv")

In [19]:
# import os
# import pandas as pd
# file_path = "./processed_csv/"
# file_list = os.listdir(file_path)
# df_concat = pd.concat([pd.read_csv(f"./processed_csv/{f}") for f in file_list], ignore_index=True)
# df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3499 entries, 0 to 3498
Columns: 102 entries, Unnamed: 0.1 to nl_text
dtypes: float64(7), int64(2), object(93)
memory usage: 2.7+ MB


In [26]:
# df_concat.sort_values("document_number",ascending=False)
# df_concat.to_csv("./folder_to_ignore/all_doc_parl_apercu_texts.csv")

,Unnamed: 0.1,Unnamed: 0,first_title,document_number,document_page_url,main_title,Unnamed: 5,Documentsprincipaux,DocumentChambre,Datededépôt,...,5.SEANCEPLENIERECHAMBRE,Mots-clÕslibres,4.SEANCEPLENIERESENAT,Annexe1,Annexe2,DatededÚp˘t,PriseenconsidÚration,CompÚtence,fr_text,nl_text
1786,NaN,0,Document parlementaire 55K3522,3522,https://www.lachambre.be/kvvcr/showpage.cfm?se...,L'évolution de la dette de l'État fédéral - Au...,NaN,00/000-K3522/001-2022/2023-0,http://www.lachambre.be/FLWB/PDF/55/3522/55K35...,04/08/2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C hambre des représentants b de elgique 4 aoû...,b K elgisChe amer van volKsvertegenwoordigers...
1787,NaN,1,Document parlementaire 55K3520,3520,https://www.lachambre.be/kvvcr/showpage.cfm?se...,Projet de loi modifiant la loi du 7 mai 1999 p...,NaN,00/000-K3520/001-2022/2023-0,http://www.lachambre.be/FLWB/PDF/55/3520/55K35...,02/08/2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C hambre des représentants b de elgique 2 aoû...,b K elgisChe amer van volKsvertegenwoordigers...
2086,NaN,2,Document parlementaire 55K3518,3518,https://www.lachambre.be/kvvcr/showpage.cfm?se...,Projet de loi introduisant le livre 2 du Code ...,NaN,00/000-K3518/001-2022/2023-0,55K3518001(pasdisponible)PROJETDELOI-CHAMBRELé...,24/07/2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PDF not disponible,PDF not disponible
1788,NaN,3,Document parlementaire 55K3516,3516,https://www.lachambre.be/kvvcr/showpage.cfm?se...,Audit de la Direction de l'information policič...,19/07/2023,00/000-K3516/001-2022/2023-0,http://www.lachambre.be/FLWB/PDF/55/3516/55K35...,19/07/2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C hambre des représentants b de elgique 19 ju...,b K elgisChe amer van volKsvertegenwoordigers...
1789,NaN,4,Document parlementaire 55K3515,3515,https://www.lachambre.be/kvvcr/showpage.cfm?se...,Projet de loi portant diverses modifications d...,NaN,00/000-K3515/001-2022/2023-0,http://www.lachambre.be/FLWB/PDF/55/3515/55K35...,20/07/2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C hambre des représentants b de elgique 20 ju...,b K elgisChe amer van volKsvertegenwoordigers...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1481,2182.0,3500,Document parlementaire 55K0008,8,https://www.lachambre.be/kvvcr/showpage.cfm?se...,"Commissions, Comitťs d'avis et dťlťgations aux...",17/12/2019,00/000-K0008/001-2019/2020-0,http://www.lachambre.be/FLWB/PDF/55/0008/55K00...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C hambre des représentants b de elgique 17 dé...,b K elgisChe amer van volKsvertegenwoordigers...
1482,2183.0,3501,Document parlementaire 55K0004,4,https://www.lachambre.be/kvvcr/showpage.cfm?se...,Liste d'adresses.,04/07/2019,00/000-K0004/001-0/2019-1,http://www.lachambre.be/FLWB/PDF/55/0004/55K00...,04/07/2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHAMBRE DES REPRÉSENTANTS DE BELGIQUE 4 juill...,BELGISCHE KAMER VAN VOLKSVERTEGENWOORDIGERS 4...
1483,2184.0,3502,Document parlementaire 55K0003,3,https://www.lachambre.be/kvvcr/showpage.cfm?se...,Composition politique.,04/07/2019,00/000-K0003/001-0/2019-1,http://www.lachambre.be/FLWB/PDF/55/0003/55K00...,04/07/2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHAMBRE DES REPRÉSENTANTS DE BELGIQUE 4 juill...,BELGISCHE KAMER VAN VOLKSVERTEGENWOORDIGERS 4...
1484,2185.0,3503,Document parlementaire 55K0002,2,https://www.lachambre.be/kvvcr/showpage.cfm?se...,Groupes linguistiques.,04/07/2019,00/000-K0002/001-0/2019-1,http://www.lachambre.be/FLWB/PDF/55/0002/55K00...,04/07/2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHAMBRE DES REPRÉSENTANTS DE BELGIQUE 4 juill...,BELGISCHE KAMER VAN VOLKSVERTEGENWOORDIGERS 4...


In [3]:
df = pd.read_json("Documents_parlementaires_récents.json")

In [5]:
df.head(2)

,Page's main url,Document Number,First title,Main title,Date de dépôt,URL of pdf,Type,Stakeholders,Commissions
0,https://www.lachambre.be/kvvcr/showpage.cfm?se...,2657,Document parlementaire 55K2657,Proposition de loi modifiant la loi du 10 nove...,28/04/2022,http://www.lachambre.be/FLWB/PDF/55/2657/55K26...,[[05 PROPOSITION DE LOI]],\n Leen Dierick cd&v \n,PRESIDENT CHAMBRE
1,https://www.lachambre.be/kvvcr/showpage.cfm?se...,3152,Document parlementaire 55K3152,Projet de loi relatif à l'approche administrat...,08/02/2023,http://www.lachambre.be/FLWB/PDF/55/3152/55K31...,[[02 PROJET DE LOI]],\n Eva Platteau Ecolo-Groen \n,PRESIDENT CHAMBRE


In [7]:
new_df = pd.read_json("New_Documents_Parlementaires_Récents.json")
new_df.T.head(2)

,text,date,pdf_link
2657,Proposition de loi modifiant la loi du 10 nove...,17/08/2023,https://www.lachambre.be/site/wwwcfm/flwb/flwb...
3152,Projet de loi relatif à l'approche administrat...,17/07/2023,https://www.lachambre.be/site/wwwcfm/flwb/flwb...
